In [ ]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
from pprint import pprint

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

# メッセージの設定
message = "言語モデルを使う上でのポイントは"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": message},
    ]
)

# レスポンスの表示
pprint(vars(response))

# 言語モデルからの回答を表示
print(response.choices[0].message.content.strip())

In [ ]:
# 役割や前提の設定
role = "あなたはマーケティング分野に精通したビジネスコンサルタントです。企業の成長をサポートするために、効果的なマーケティング戦略を提供します。"

# メッセージの設定
message = "新製品の発売に向けた効果的なマーケティング戦略を教えてください。"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": message},
    ],
)

# 言語モデルからの回答を表示
print(response.choices[0].message.content.strip())

In [ ]:
# 出力の多様性
message = "日本の魅力を紹介して！"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": message},
    ],
    temperature=1.4,
    n = 3, # 3回答を得る
    max_completion_tokens = 100 # 出力トークン長を制限
)

# 結果を表示（複数回答を比較）
for choice in response.choices:
    print("-" * 20)
    print(choice.message.content.strip())

In [ ]:
import tiktoken
from tiktoken.core import Encoding

# OpenAI APIの特定のモデルに対応するトークナイザーを取得
encoding: Encoding = tiktoken.encoding_for_model(MODEL_NAME) # o200k_base

# テキストをトークンIDのリストに変換
tokens = encoding.encode("こんにちは！")
tokens_count = len(tokens)

# トークンの長さとトークンIDを表示
print(f"{tokens_count=}")
print(f"{tokens=}")

In [ ]:
# トークンの選択度合い
message = """
AさんとBさんで会話してください。
A:あ、Bさんだ。
B:
"""

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": message},
    ],
    max_tokens=100,
    n=3,
    logit_bias = {95839:8, 3393:8}
)

# 結果を表示
for choice in response.choices:
    print("-" * 20)
    print(choice.message.content.strip())

In [ ]:
# メッセージの設定
message = "言語モデルを使う上でのポイントは"

# APIへリクエスト
stream = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": message},
    ],
    stream=True,
)

for chunk in stream:
    if chunk.choices:
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end="")